In [1]:
# importing modules
import numpy as np
import pandas as pd

In [2]:
import time
import plotly as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
#import plot_settings
import math

In [3]:
np.set_printoptions(threshold=np.inf)
np.random.seed(42)

[Jump to Final Output](#Final_Output)<br>
[Jump to List of Companies](#List)<br>
[R&D Multiple Companies](#R&D)<br>
[Plotting](#Plotting)

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
working_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2021/'

In [6]:
roic_import = pd.read_csv('{}{}'\
.format(working_dir,'New_ROIC_Historical--IWV-2016-2020-acq-nr.csv'), low_memory=False,
na_values=['#VALUE!', '#DIV/0!'])

In [7]:
# roic_import.columns

In [8]:
# roic_import = roic_import[roic_import['StartDate'] == '12/31/2014']

In [9]:
roic_import['Instance'] = roic_import['Symbol'].astype(str) + '_' + roic_import['StartDate'].astype(str)

In [10]:
roic_import = roic_import.set_index('Instance')

In [11]:
#roic_import.columns.values

In [12]:
roic_import.head(2)

,Symbol,Company_Name,StartDate,EndDate,AdjEndDate,Total_Return,Price,MarketCap,EnterpriseValue,Sales,...,ShareholderYield1,ShareholderYield2,Sector,Cash.1,Investments,Cash+LT,ST_Debt,LT_Debt,Net_Cash,R_D
Instance,,,,,,,,,,,,,,,,,,,,,
AAPL-US_12/31/2019,AAPL-US,Apple Inc.,12/31/2019,12/31/2020,12/31/2020,81.845730,73.4125,1287643.1,1295110.1,259968.0,...,0.080379,0.061563,Information Technology,90580.00,132366.0,222946.00,20973.0,89983.25,111989.75,16217.0
MSFT-US_12/31/2019,MSFT-US,Microsoft Corporation,12/31/2019,12/31/2020,12/31/2020,42.365253,157.7000,1200254.6,1149592.6,129565.0,...,0.018010,0.028233,Information Technology,132433.75,2502.5,134936.25,6440.0,79623.75,48872.50,17464.0


In [13]:
roic_import.tail(2)

,Symbol,Company_Name,StartDate,EndDate,AdjEndDate,Total_Return,Price,MarketCap,EnterpriseValue,Sales,...,ShareholderYield1,ShareholderYield2,Sector,Cash.1,Investments,Cash+LT,ST_Debt,LT_Debt,Net_Cash,R_D
Instance,,,,,,,,,,,,,,,,,,,,,
SHOP-US_12/31/2016,SHOP-US,"Shopify, Inc. Class A",12/31/2016,12/31/2017,12/29/2017,135.59600,42.87,3832.8127,3432.5260,328.554,...,-0.143685,-0.060872,NaN,256.539333,0.00000,256.539333,0.0,0.0,256.539333,63.940
SHOP-US_12/31/2015,SHOP-US,"Shopify, Inc. Class A",12/31/2015,12/31/2016,12/30/2016,66.16279,25.80,2066.3184,1879.3823,170.242,...,NaN,NaN,NaN,126.148750,1.13625,127.285000,0.0,0.0,127.285000,33.898


In [14]:
roic_import.shape

(14854, 68)

In [15]:
# roic_filtered.dtypes

In [16]:
# just have one class of google shares
roic_import = roic_import[roic_import['Symbol'] != 'GOOGL-US']

In [17]:
print('BEFORE:', roic_import.isnull().values.any())   # prints True

# fill missing values with zeros
roic_filtered = roic_import.fillna(0)

print('AFTER:',roic_filtered.isnull().values.any())   # prints False now :)

BEFORE: True
AFTER: False


In [18]:
print('ORIGINAL:',roic_import.isnull().values.any())   # Should still print True if we didn't change original

ORIGINAL: True


In [19]:
# roic_filtered = roic_import.fillna(0)

In [20]:
roic_filtered.shape

(14849, 68)

In [21]:
# set the index to instance

# roic_filtered = roic_filtered.set_index('Instance')

## Beta import

In [22]:
# import the table of symbols and sectors

data_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2021/'
betadf = pd.read_csv('{}{}'\
.format(data_dir,'Upside Downside Trailing Beta - IWV - 2015-2019.csv'), low_memory=False,
                   na_values=['#VALUE!', '#DIV/0!'])
betadf = betadf[['Instance', 'Beta','UpsideBeta','DownsideBeta']]

In [23]:
# merge the beta lookup into the roic filtered dataframe

roic_filtered = roic_filtered.reset_index()
roic_filtered = pd.merge(roic_filtered,betadf, on='Instance')
roic_filtered = roic_filtered.set_index('Instance')

In [24]:
roic_filtered['UpsideSpread'] = roic_filtered.UpsideBeta - roic_filtered.DownsideBeta

In [25]:
roic_filtered.shape

(14843, 72)

In [26]:
roic_filtered = roic_filtered.drop_duplicates()

In [27]:
roic_filtered.shape

(14843, 72)

## Momo import

In [28]:
#### merging in momentum data

In [29]:
# import the momentum data
momo2021df = pd.read_csv('{}{}'.format(working_dir,'Momentum_Lookup_Subtract_IWV_2016_2020.csv'), 
                         low_memory=False, na_values=['#VALUE!', '#DIV/0!'])

momo2021df = momo2021df[['Instance','3mo_Return','6mo_Return','1year_Return','relative_3mo_spy',
                 'relative_6mo_spy','relative_1y_spy','relative_3mo_ijr','relative_6mo_ijr',
                 'relative_1y_ijr','relative_3mo_xlp','relative_6mo_xlp','relative_1y_xlp']]
momo2021df = momo2021df.fillna(0)

In [30]:
# merge the beta lookup into the roic filtered dataframe
roic_filtered = roic_filtered.reset_index()
roic_filtered = pd.merge(roic_filtered,momo2021df, on='Instance')
roic_filtered = roic_filtered.set_index('Instance')

In [31]:
roic_filtered.shape

(14843, 84)

## Manually created features

In [32]:
# make some new features with some simple division and addition

In [33]:
roic_filtered['Net_Cash_Pct'] = roic_filtered.Net_Cash / roic_filtered.MarketCap

In [34]:
roic_filtered['EBIT_RD'] = roic_filtered.EBIT + roic_filtered.R_D
roic_filtered['OCF_RD'] = roic_filtered.OCF + roic_filtered.R_D
roic_filtered['FCF_RD'] = roic_filtered.FCF + roic_filtered.R_D

In [35]:
roic_filtered['EBIT_RD_ROIC'] = roic_filtered.EBIT_RD / roic_filtered.TangibleCapital
roic_filtered['OCF_RD_ROIC'] = roic_filtered.OCF_RD / roic_filtered.TangibleCapital
roic_filtered['FCF_RD_ROIC'] = roic_filtered.FCF_RD / roic_filtered.TangibleCapital

In [36]:
roic_filtered['RD_Cap'] = roic_filtered.R_D / roic_filtered.TangibleCapital
roic_filtered['RD_Sales'] = roic_filtered.R_D / roic_filtered.Sales

In [37]:
roic_filtered['EBIT_RD_EV'] = roic_filtered.EBIT_RD / roic_filtered.EnterpriseValue
roic_filtered['OCF_RD_EV'] = roic_filtered.OCF_RD / roic_filtered.EnterpriseValue
roic_filtered['FCF_RD_EV'] = roic_filtered.FCF_RD / roic_filtered.EnterpriseValue

In [38]:
roic_filtered.shape

(14843, 96)

In [39]:
roic_filtered['Price_Sales'] = roic_filtered.MarketCap / roic_filtered.Sales

In [40]:
roic_filtered = roic_filtered.rename(columns={'R_D':'RD'})

In [41]:
# change any infinity or negative infinity values to NaN
roic_filtered = roic_filtered.replace([np.inf, -np.inf], np.nan)

In [42]:
# fill NaNs with zeros
roic_filtered = roic_filtered.fillna(0)

In [43]:
# roic_filtered.head()

## Prior Year Import

In [44]:
roic_filtered = roic_filtered.reset_index().drop(columns='Instance')

In [45]:
# roic_filtered.StartDate.replace('12/9/2020','12/31/2020', inplace=True)24

In [46]:
roic_filtered['Instance'] = roic_filtered['Symbol'].astype(str) + '_' + roic_filtered['StartDate'].astype(str)
roic_filtered = roic_filtered.set_index('Instance')

In [47]:
# priordir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2020/'
# prioryear = pd.read_csv('{}{}'.format(priordir,'ROIC_2020_with_Deltas_and_beta_through_2020_12_24.csv'),
#                         low_memory=False)


priordir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2021/'
prioryear = pd.read_csv('{}{}'.format(priordir,'New_ROIC_Historical--IWV-2020-acq-nr-one-day-offset.csv'),
                        low_memory=False, na_values=['#VALUE!', '#DIV/0!'])

In [48]:
prioryear = prioryear[prioryear['StartDate'] == '12/31/2020']
prioryear['Instance'] = prioryear['Symbol'].astype(str) + '_' + prioryear['StartDate'].astype(str)
prioryear = prioryear.set_index('Instance')
prioryear.shape

(2858, 68)

In [49]:
prioryear['Net_Cash_Pct'] = prioryear.Net_Cash / prioryear.MarketCap

prioryear['EBIT_RD'] = prioryear.EBIT + prioryear.R_D
prioryear['OCF_RD'] = prioryear.OCF + prioryear.R_D
prioryear['FCF_RD'] = prioryear.FCF + prioryear.R_D

prioryear['EBIT_RD_ROIC'] = prioryear.EBIT_RD / prioryear.TangibleCapital
prioryear['OCF_RD_ROIC'] = prioryear.OCF_RD / prioryear.TangibleCapital
prioryear['FCF_RD_ROIC'] = prioryear.FCF_RD / prioryear.TangibleCapital

prioryear['RD_Cap'] = prioryear.R_D / prioryear.TangibleCapital
prioryear['RD_Sales'] = prioryear.R_D / prioryear.Sales

prioryear['Price_Sales'] = prioryear.MarketCap / prioryear.Sales

prioryear['EBIT_RD_EV'] = prioryear.EBIT_RD / prioryear.EnterpriseValue
prioryear['OCF_RD_EV'] = prioryear.OCF_RD / prioryear.EnterpriseValue
prioryear['FCF_RD_EV'] = prioryear.FCF_RD / prioryear.EnterpriseValue

In [50]:
# import the table of symbols and sectors

data_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2021/'
betadf = pd.read_csv('{}{}'\
.format(data_dir,'Upside Downside Trailing Beta - IWV - 2020 - 20201231.csv'), 
                     low_memory=False, na_values=['#VALUE!', '#DIV/0!'])
betadf = betadf[['Instance', 'Beta','UpsideBeta','DownsideBeta']]

In [51]:
# merge the beta lookup into the roic filtered dataframe

prioryear = prioryear.reset_index()
prioryear = pd.merge(prioryear,betadf, on='Instance')
prioryear = prioryear.set_index('Instance')

In [52]:
prioryear = prioryear.rename(columns={'R_D':'RD'})

In [53]:
combined = pd.concat([roic_filtered,prioryear], sort=True)

In [54]:
combined = combined.fillna(0)

In [55]:
combined.shape

(17700, 97)

In [56]:
roic_filtered.shape

(14843, 97)

In [57]:
roic_import.shape

(14849, 68)

### Import for 2022

In [58]:
working_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2022/'

import_2022 = pd.read_csv('{}{}'\
.format(working_dir,'New_ROIC_Historical--IWV-2021-acq-nr-20211231-dpz-fix.csv'), low_memory=False,
na_values=['#VALUE!', '#DIV/0!','#SPILL!'])

In [59]:
import_2022['Instance'] = import_2022['Symbol'].astype(str) + '_' + import_2022['StartDate'].astype(str)

In [60]:
import_2022.head(2)

,Symbol,Company_Name,StartDate,EndDate,AdjEndDate,Total_Return,Price,MarketCap,EnterpriseValue,Sales,...,ShareholderYield2,Sector,Cash.1,Investments,Cash+LT,ST_Debt,LT_Debt,Net_Cash,R_D,Instance
0,MSFT-US,Microsoft Corporation,12/31/2021,12/31/2021,12/31/2021,0.0,336.32,2525084.0,2473404.0,176251.0,...,0.017895,Information Technology,129787.80,5638.0,135425.80,8363.40,72474.6,54587.8,21389.0,MSFT-US_12/31/2021
1,AAPL-US,Apple Inc.,12/31/2021,12/31/2021,12/31/2021,0.0,177.57,2913284.8,2987167.8,366295.0,...,0.034086,Information Technology,67748.75,129396.0,197144.75,14736.25,108264.0,74144.5,21914.0,AAPL-US_12/31/2021


In [61]:
# fill missing values with zeros
import_2022 = import_2022.fillna(0)

In [62]:
# make some new features with some simple division and addition

import_2022['Net_Cash_Pct'] = import_2022.Net_Cash / import_2022.MarketCap

import_2022['EBIT_RD'] = import_2022.EBIT + import_2022.R_D
import_2022['OCF_RD'] = import_2022.OCF + import_2022.R_D
import_2022['FCF_RD'] = import_2022.FCF + import_2022.R_D

import_2022['EBIT_RD_ROIC'] = import_2022.EBIT_RD / import_2022.TangibleCapital
import_2022['OCF_RD_ROIC'] = import_2022.OCF_RD / import_2022.TangibleCapital
import_2022['FCF_RD_ROIC'] = import_2022.FCF_RD / import_2022.TangibleCapital

import_2022['RD_Cap'] = import_2022.R_D / import_2022.TangibleCapital
import_2022['RD_Sales'] = import_2022.R_D / import_2022.Sales

import_2022['EBIT_RD_EV'] = import_2022.EBIT_RD / import_2022.EnterpriseValue
import_2022['OCF_RD_EV'] = import_2022.OCF_RD / import_2022.EnterpriseValue
import_2022['FCF_RD_EV'] = import_2022.FCF_RD / import_2022.EnterpriseValue

import_2022.shape

import_2022['Price_Sales'] = import_2022.MarketCap / import_2022.Sales

import_2022 = import_2022.rename(columns={'R_D':'RD'})

# change any infinity or negative infinity values to NaN
import_2022 = import_2022.replace([np.inf, -np.inf], np.nan)

(2717, 81)

In [63]:
import_2022 = import_2022.set_index('Instance')

In [64]:
# import the table of symbols and sectors

data_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2022/'
betadf = pd.read_csv('{}{}'\
.format(data_dir,'Upside Downside Trailing Beta - IWV - 2021 - 20211231.csv'), 
                     low_memory=False, na_values=['#VALUE!', '#DIV/0!','#N/A'])
betadf = betadf[['Instance', 'Beta','UpsideBeta','DownsideBeta']]

betadf = betadf.fillna(0)

# merge the beta lookup into the roic filtered dataframe

import_2022 = import_2022.reset_index()
import_2022 = pd.merge(import_2022,betadf, on='Instance')
import_2022 = import_2022.set_index('Instance')

In [65]:
combined = pd.concat([combined,import_2022], sort=True)

In [66]:
combined.shape

(20416, 97)

### Import for LTM

In [67]:
working_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2022/'

import_LTM = pd.read_csv('{}{}'\
.format(working_dir,'New_ROIC_Historical--IWV-dpz-fix-shop-LTM-20220506.csv'), low_memory=False,
na_values=['#VALUE!', '#DIV/0!','#SPILL!'])

In [68]:
import_LTM['Instance'] = import_LTM['Symbol'].astype(str) + '_' + import_LTM['StartDate'].astype(str)

In [69]:
import_LTM.head(2)

,Symbol,Company_Name,StartDate,EndDate,AdjEndDate,Total_Return,Price,MarketCap,EnterpriseValue,Sales,...,ShareholderYield2,Sector,Cash.1,Investments,Cash+LT,ST_Debt,LT_Debt,Net_Cash,R_D,Instance
0,MSFT-US,Microsoft Corporation,5/6/2022,5/6/2022,05/06/2022,0.0,274.73,2054713.9,2028001.9,192557.0,...,0.023280,Information Technology,122752.75,6594.00,129346.75,7492.25,72394.50,49460.0,23350.0,MSFT-US_5/6/2022
1,AAPL-US,Apple Inc.,5/6/2022,5/6/2022,05/06/2022,0.0,157.28,2566715.3,2635185.3,386077.0,...,0.039049,Information Technology,59939.75,135238.25,195178.00,16501.75,108771.25,69905.0,24182.0,AAPL-US_5/6/2022


In [70]:
# fill missing values with zeros
import_LTM = import_LTM.fillna(0)

In [71]:
# make some new features with some simple division and addition

import_LTM['Net_Cash_Pct'] = import_LTM.Net_Cash / import_LTM.MarketCap

import_LTM['EBIT_RD'] = import_LTM.EBIT + import_LTM.R_D
import_LTM['OCF_RD'] = import_LTM.OCF + import_LTM.R_D
import_LTM['FCF_RD'] = import_LTM.FCF + import_LTM.R_D

import_LTM['EBIT_RD_ROIC'] = import_LTM.EBIT_RD / import_LTM.TangibleCapital
import_LTM['OCF_RD_ROIC'] = import_LTM.OCF_RD / import_LTM.TangibleCapital
import_LTM['FCF_RD_ROIC'] = import_LTM.FCF_RD / import_LTM.TangibleCapital

import_LTM['RD_Cap'] = import_LTM.R_D / import_LTM.TangibleCapital
import_LTM['RD_Sales'] = import_LTM.R_D / import_LTM.Sales

import_LTM['EBIT_RD_EV'] = import_LTM.EBIT_RD / import_LTM.EnterpriseValue
import_LTM['OCF_RD_EV'] = import_LTM.OCF_RD / import_LTM.EnterpriseValue
import_LTM['FCF_RD_EV'] = import_LTM.FCF_RD / import_LTM.EnterpriseValue

import_LTM.shape

import_LTM['Price_Sales'] = import_LTM.MarketCap / import_LTM.Sales

import_LTM = import_LTM.rename(columns={'R_D':'RD'})

# change any infinity or negative infinity values to NaN
import_LTM = import_LTM.replace([np.inf, -np.inf], np.nan)

(2717, 81)

In [72]:
import_LTM = import_LTM.set_index('Instance')

In [73]:
# import the table of symbols and sectors

data_dir = '/Users/evan/Library/Mobile Documents/com~apple~CloudDocs/Data2022/'
betadf = pd.read_csv('{}{}'\
.format(data_dir,'Upside Downside Trailing Beta - IWV - LTM - 20220506.csv'), 
                     low_memory=False, na_values=['#VALUE!', '#DIV/0!','#N/A'])
betadf = betadf[['Instance', 'Beta','UpsideBeta','DownsideBeta']]

betadf = betadf.fillna(0)

# merge the beta lookup into the roic filtered dataframe

import_LTM = import_LTM.reset_index()
import_LTM = pd.merge(import_LTM,betadf, on='Instance')
import_LTM = import_LTM.set_index('Instance')

In [74]:
combined = pd.concat([combined,import_LTM], sort=True)

In [75]:
combined.shape

(23132, 97)

In [76]:
import_LTM.shape

(2716, 84)

In [77]:
betadf.shape

(2717, 4)

In [78]:
# combined.columns

### Calculate Deltas with built-in vectorized functions

In [79]:
#### make a subset dataframe to workout for calculating the deltas

deltadf = combined[['Symbol', 'StartDate', 'EBIT_ROIC','OCF_ROIC','FCF_ROIC','TangibleCapital']].\
                    sort_values(by=['Symbol','StartDate'])

In [80]:
#### calculate the change in ROIC from the prior year. Simple subtraction for each 
#### Symbol compared to its prior year value
#### Calculate the growth rate in tangible capital yoy for each company

import time
ta = time.time()

deltadf = deltadf.assign(ebit_prior = lambda t: t.groupby('Symbol')['EBIT_ROIC'].shift(1),
                         ebit_chg = lambda t: t['EBIT_ROIC'] - t['ebit_prior'],
                         ocf_prior = lambda t: t.groupby('Symbol')['OCF_ROIC'].shift(1),
                         ocf_chg = lambda t: t['OCF_ROIC'] - t['ocf_prior'],
                         fcf_prior = lambda t: t.groupby('Symbol')['FCF_ROIC'].shift(1),
                         fcf_chg = lambda t: t['FCF_ROIC'] - t['fcf_prior'],
                         tangible_pctchg = lambda t: t.groupby('Symbol')['TangibleCapital'].pct_change())

tb = time.time()
totalab = tb-ta
print(round(totalab,2),'seconds')

0.03 seconds


In [81]:
print('''If you want to handle the companies that have data every 2, 3, 4... years
and not every single year, I can write logic to handle that. I could input rows for the
missing years, populate them with 0 or something, and then do the calculations, but that 
could be misleading.''')

If you want to handle the companies that have data every 2, 3, 4... years
and not every single year, I can write logic to handle that. I could input rows for the
missing years, populate them with 0 or something, and then do the calculations, but that 
could be misleading.


In [82]:
# deltadf.head(8)

In [83]:
# deltadf[deltadf.Symbol=='MSFT-US']

In [84]:
deltadf[deltadf.Symbol=='AAPL-US']

,Symbol,StartDate,EBIT_ROIC,OCF_ROIC,FCF_ROIC,TangibleCapital,ebit_prior,ebit_chg,ocf_prior,ocf_chg,fcf_prior,fcf_chg,tangible_pctchg
Instance,,,,,,,,,,,,,
AAPL-US_12/31/2015,AAPL-US,12/31/2015,2.037599,2.485180,2.133867,32700.25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAPL-US_12/31/2016,AAPL-US,12/31/2016,1.900757,2.170523,1.723782,30326.33333,2.037599,-0.136842,2.485180,-0.314657,2.133867,-0.410085,-0.072596
AAPL-US_12/31/2017,AAPL-US,12/31/2017,1.752080,1.856280,1.482823,34261.00000,1.900757,-0.148678,2.170523,-0.314243,1.723782,-0.240959,0.129744
AAPL-US_12/31/2018,AAPL-US,12/31/2018,1.622862,1.768879,1.464761,43775.75000,1.752080,-0.129218,1.856280,-0.087401,1.482823,-0.018062,0.277714
AAPL-US_12/31/2019,AAPL-US,12/31/2019,1.246664,1.367876,1.160993,50729.00000,1.622862,-0.376198,1.768879,-0.401003,1.464761,-0.303768,0.158838
AAPL-US_12/31/2020,AAPL-US,12/31/2020,1.152953,1.423550,1.294577,56671.00000,1.246664,-0.093711,1.367876,0.055674,1.160993,0.133585,0.117132
AAPL-US_12/31/2021,AAPL-US,12/31/2021,2.016269,1.903889,1.701034,54645.00000,1.152953,0.863316,1.423550,0.480339,1.294577,0.406456,-0.035750
AAPL-US_5/6/2022,AAPL-US,5/6/2022,2.122241,2.066957,1.878185,56327.25000,2.016269,0.105972,1.903889,0.163068,1.701034,0.177151,0.030785


In [85]:
#### Rename the columns to what I usually call these

deltadf = deltadf.rename(columns={'ebit_chg':'EBIT_ROIC_Delta','ocf_chg':'OCF_ROIC_Delta',
                       'fcf_chg':'FCF_ROIC_Delta','tangible_pctchg':'Capital_Growth'})

In [86]:
#### what's the change in ROIC relative to the base value of ROIC?

deltadf['EBIT_ROIC_rel'] = deltadf['EBIT_ROIC_Delta'] / deltadf['ebit_prior'] 
deltadf['OCF_ROIC_rel'] = deltadf['OCF_ROIC_Delta'] / deltadf['ocf_prior'] 
deltadf['FCF_ROIC_rel'] = deltadf['FCF_ROIC_Delta'] / deltadf['fcf_prior'] 

In [87]:
#### If a prior period's profits are negative, and the following year is positive,
#### you get a negative growth rate calculated, even those it's massive positive growth.
#### while flipping the sign isn't the perfect solution, the result is much closer
#### to the ground truth of the situation than the straight calculation, which leads one
#### in the exact opposite direction (where big improvement calcs to big negative growth rate)

deltadf['EBIT_ROIC_rel'] = np.where((deltadf.ebit_prior < 0) & (deltadf.EBIT_ROIC > 0), 
                                    deltadf['EBIT_ROIC_rel'] * -1, deltadf['EBIT_ROIC_rel'])

deltadf['OCF_ROIC_rel'] = np.where((deltadf.ocf_prior < 0) & (deltadf.OCF_ROIC > 0), 
                                    deltadf['OCF_ROIC_rel'] * -1, deltadf['OCF_ROIC_rel'])

deltadf['FCF_ROIC_rel'] = np.where((deltadf.fcf_prior < 0) & (deltadf.FCF_ROIC > 0), 
                                    deltadf['FCF_ROIC_rel'] * -1, deltadf['FCF_ROIC_rel'])

In [88]:
deltadf = deltadf.drop(['ebit_prior','ocf_prior','fcf_prior'], axis=1)

In [89]:
#### Replace any empty values with NaN and then fill NaNs with zeros

list_of_changes = ['EBIT_ROIC_Delta', 'OCF_ROIC_Delta','FCF_ROIC_Delta','Capital_Growth',
                  'EBIT_ROIC_rel','OCF_ROIC_rel','FCF_ROIC_rel']

for i in (list_of_changes):
    deltadf[i] = deltadf[i].replace('', np.nan, regex=True)
    deltadf[i] = deltadf[i].fillna(0)

# deltadf['EBIT_ROIC_Delta'] = deltadf['EBIT_ROIC_Delta'].replace('', np.nan, regex=True)
# deltadf['EBIT_ROIC_Delta'] = deltadf['EBIT_ROIC_Delta'].fillna(0)
#

In [90]:
#### Merge the delta dataframe with the main dataframe from before this section

deltadf = deltadf.reset_index()
deltadf = deltadf[['Instance','EBIT_ROIC_Delta','OCF_ROIC_Delta','FCF_ROIC_Delta','Capital_Growth',
                   'EBIT_ROIC_rel','OCF_ROIC_rel','FCF_ROIC_rel']]

roic_with_deltas = combined.reset_index()
roic_with_deltas = pd.merge(roic_with_deltas,deltadf, on='Instance')
roic_with_deltas = roic_with_deltas.set_index('Instance')

# roic_with_deltas.shape

# Lookup a Single Company and Track LTM and Last 5 Years

In [91]:
# roic_pretty = roic_filtered.copy()

In [92]:
# roic_pretty = combined.copy()

In [93]:
roic_pretty = roic_with_deltas.copy()

In [94]:
pd.set_option('display.max_rows', 500)

In [95]:
# roic_pretty.dtypes

In [96]:
roic_pretty = roic_pretty.drop('Cash.1', axis=1)

In [97]:
roic_pretty = roic_pretty.reset_index()

In [98]:
# roic_pretty = pd.concat([roic_pretty,roic_2016_2020], sort=True)

In [99]:
roic_pretty = roic_pretty.set_index('Instance')

In [100]:
## use a dictionary to define style formatting. it's more elegant than the attempt above

format_dict = {'MarketCap':'{:,.0f}', 'Net_Cash':'{:,.0f}', 'R_D':'{:,.0f}',
              'FCF_ROIC':'{:,.1%}', 'EBIT_ROIC':'{:,.1%}', 'ShareholderYield1':'{:,.1%}', 
              'DividendYield':'{:,.1%}','EBIT_EV':'{:,.1%}','FCF_RD_ROIC':'{:,.1%}',
              'DownsideBeta3yr':'{:.2f}','Total_Return':'{:.2f}','RD_Cap':'{:.2f}',
              'EBITGrowth5year':'{:,.1%}'}

In [101]:
list_of_whole_numbers = ['Sales', 'EBIT', 'OCF', 'CurrentAssets', 'Cash', 'LT_Debt', 'AccountsPayable',
                        'NetFixedAssets', 'TangibleCapital', 'RD', 'FCF','MarketCap','Capex','OCF_RD']

list_of_percentages = ['EBIT_ROIC', 'OCF_ROIC', 'ROA', 'FCF_ROIC', 'FCF_RD_ROIC', 'RD_Cap','RD_Sales',
                      'ShareholderYield1', 'DividendYield', 'EBIT_EV','OCF_EV','FCF_EV','EBIT_RD_EV',
                       'OCF_RD_EV','FCF_RD_EV', 'OCF_RD_ROIC', 'EBITGrowth1year', 'EBITGrowth3year',
                       'EBITGrowth5year', 'SalesGrowth1year','SalesGrowth3year', 'SalesGrowth5year',
                       'OCFGrowth1year','OCFGrowth3year', 'OCFGrowth5year',
                       'EBIT_ROIC_Delta','OCF_ROIC_Delta','FCF_ROIC_Delta','Capital_Growth',
                      'EBIT_ROIC_rel','OCF_ROIC_rel','FCF_ROIC_rel']

list_of_floats = ['DownsideBeta', 'UpsideBeta','Beta']

for i in list_of_whole_numbers:
    roic_pretty[i] = roic_pretty[i].round(0).astype(int).map("{:,}".format)
    
for j in list_of_percentages:
    roic_pretty[j] = roic_pretty[j].astype(float).map("{:,.1%}".format)
    
for f in list_of_floats:
    roic_pretty[f] = roic_pretty[f].astype(float).map('{0:.2f}'.format)
    
#'{0:.2f}'
# "{:,.3}"

In [102]:
roic_pretty['StartDate'] = pd.to_datetime(roic_pretty['StartDate']) 

In [103]:
# roic_pretty[roic_pretty.Symbol == 'NFLX-US'].sort_values('StartDate', ascending=False)\
# [['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA','CurrentAssets','Cash','LT_Debt','AccountsPayable', 
# 'NetFixedAssets', 'TangibleCapital']].T

In [104]:
# roic_pretty[roic_pretty.Symbol == 'NFLX-US'].sort_values('StartDate', ascending=False)\
# [['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA','CurrentAssets','Cash','LT_Debt','AccountsPayable', 
# 'NetFixedAssets', 'TangibleCapital']].head().T

In [105]:
roic_pretty.shape

(23132, 103)

In [106]:
roic_pretty = roic_pretty.rename(columns={'AccountsPayable':'ActPay', 'CurrentAssets':'CurrAssets',
                                         'NetFixedAssets':'NetFixed', 'EBITGrowth1year':'EBITGrowth-1y', 
                                          'EBITGrowth3year':'EBITGrowth-3y', 'SalesGrowth1year':'SalesGrowth-1y',
                                          'SalesGrowth3year':'SalesGrowth-3y','OCFGrowth1year':'OCFGrowth-1y',
                                          'OCFGrowth3year':'OCFGrowth-3y'})

In [107]:
# roic_pretty.columns

In [108]:
# import pickle
# roic_pretty.to_pickle('roic_r3k_2022-deltas-ltm-20220506.pkl')

In [109]:
# roic_pretty[roic_pretty.StartDate=='12-3-2021'][['Sales','EBIT','OCF']].head(20)

In [110]:
# roic_pretty.loc[roic_pretty['Company_Name'].str.contains('Ford',True)]['Symbol'].head(1)[0]

<a id='Final_Output'></a>

## Type in Symbol

In [111]:
lookup_symbol = 'PG-US'

In [112]:
roic_pretty[(roic_pretty.Symbol==lookup_symbol)]\
[['Company_Name']].tail(1)

,Company_Name
Instance,
PG-US_5/6/2022,Procter & Gamble Company


In [113]:
roic_pretty[(roic_pretty.Symbol==lookup_symbol)]\
.sort_values('StartDate', ascending=False)\
[['MarketCap']]

,MarketCap
Instance,
PG-US_5/6/2022,"374,291"
PG-US_12/31/2021,"395,855"
PG-US_12/31/2020,"345,012"
PG-US_12/31/2019,"308,435"
PG-US_12/31/2018,"229,945"
PG-US_12/31/2017,"231,630"
PG-US_12/31/2016,"214,949"
PG-US_12/31/2015,"214,770"


# Results

In [114]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA',
'CurrAssets','Cash','LT_Debt','ActPay','NetFixed', 'TangibleCapital']]

,Sales,EBIT,EBIT_ROIC,OCF,OCF_ROIC,ROA,CurrAssets,Cash,LT_Debt,ActPay,NetFixed,TangibleCapital
Instance,,,,,,,,,,,,
PG-US_5/6/2022,"79,618","18,244",84.9%,"17,131",79.7%,12.1%,"24,097","10,182","22,594","14,068","21,642","21,488"
PG-US_12/31/2021,"77,138","18,353",88.5%,"18,275",85.0%,11.9%,"23,805","10,595","21,683","13,265","21,559","21,504"
PG-US_12/31/2020,"72,470","16,984",75.9%,"17,973",80.4%,11.6%,"25,018","12,811","22,938","11,313","21,471","22,365"
PG-US_12/31/2019,"68,792","15,290",66.6%,"15,844",69.0%,3.6%,"22,785","10,383","21,015","10,671","21,218","22,950"
PG-US_12/31/2018,"66,869","14,579",64.1%,"14,803",65.1%,8.3%,"26,460","14,349","21,566","10,011","20,634","22,734"
PG-US_12/31/2017,"65,193","14,512",64.7%,"13,359",59.6%,8.5%,"26,512","14,731","17,830","8,866","19,500","22,414"
PG-US_12/31/2016,"65,290","14,552",47.5%,"14,922",48.7%,7.8%,"34,423","14,369","18,996","8,715","19,294","30,633"
PG-US_12/31/2015,"72,620","15,394",47.6%,"14,513",44.8%,6.7%,"32,502","12,407","17,803","7,758","20,034","32,370"


## Deltas

In [115]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['OCF', 'OCFGrowth-1y', 'OCF_ROIC', 'OCF_ROIC_Delta', 'TangibleCapital', 'Capital_Growth']]

,OCF,OCFGrowth-1y,OCF_ROIC,OCF_ROIC_Delta,TangibleCapital,Capital_Growth
Instance,,,,,,
PG-US_5/6/2022,"17,131",-10.1%,79.7%,-5.3%,"21,488",-0.1%
PG-US_12/31/2021,"18,275",1.7%,85.0%,4.6%,"21,504",-3.8%
PG-US_12/31/2020,"17,973",13.4%,80.4%,11.3%,"22,365",-2.6%
PG-US_12/31/2019,"15,844",7.0%,69.0%,3.9%,"22,950",0.9%
PG-US_12/31/2018,"14,803",10.8%,65.1%,5.5%,"22,734",1.4%
PG-US_12/31/2017,"13,359",-10.5%,59.6%,10.9%,"22,414",-26.8%
PG-US_12/31/2016,"14,922",2.8%,48.7%,3.9%,"30,633",-5.4%
PG-US_12/31/2015,"14,513",-6.7%,44.8%,0.0%,"32,370",0.0%


In [116]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['OCF', 'OCFGrowth-1y', 'OCF_ROIC', 'OCF_ROIC_Delta', 'OCF_ROIC_rel', 'TangibleCapital', 'Capital_Growth']]

#  'EBIT_ROIC_rel','OCF_ROIC_rel','FCF_ROIC_rel'

,OCF,OCFGrowth-1y,OCF_ROIC,OCF_ROIC_Delta,OCF_ROIC_rel,TangibleCapital,Capital_Growth
Instance,,,,,,,
PG-US_5/6/2022,"17,131",-10.1%,79.7%,-5.3%,-6.2%,"21,488",-0.1%
PG-US_12/31/2021,"18,275",1.7%,85.0%,4.6%,5.8%,"21,504",-3.8%
PG-US_12/31/2020,"17,973",13.4%,80.4%,11.3%,16.4%,"22,365",-2.6%
PG-US_12/31/2019,"15,844",7.0%,69.0%,3.9%,6.0%,"22,950",0.9%
PG-US_12/31/2018,"14,803",10.8%,65.1%,5.5%,9.2%,"22,734",1.4%
PG-US_12/31/2017,"13,359",-10.5%,59.6%,10.9%,22.4%,"22,414",-26.8%
PG-US_12/31/2016,"14,922",2.8%,48.7%,3.9%,8.6%,"30,633",-5.4%
PG-US_12/31/2015,"14,513",-6.7%,44.8%,0.0%,0.0%,"32,370",0.0%


In [117]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['OCF', 'OCFGrowth-1y', 'EBIT_ROIC', 'EBIT_ROIC_Delta', 'EBIT_ROIC_rel', 'TangibleCapital', 'Capital_Growth']]

#  'EBIT_ROIC_rel','OCF_ROIC_rel','FCF_ROIC_rel'

,OCF,OCFGrowth-1y,EBIT_ROIC,EBIT_ROIC_Delta,EBIT_ROIC_rel,TangibleCapital,Capital_Growth
Instance,,,,,,,
PG-US_5/6/2022,"17,131",-10.1%,84.9%,-3.6%,-4.0%,"21,488",-0.1%
PG-US_12/31/2021,"18,275",1.7%,88.5%,12.5%,16.5%,"21,504",-3.8%
PG-US_12/31/2020,"17,973",13.4%,75.9%,9.3%,14.0%,"22,365",-2.6%
PG-US_12/31/2019,"15,844",7.0%,66.6%,2.5%,3.9%,"22,950",0.9%
PG-US_12/31/2018,"14,803",10.8%,64.1%,-0.6%,-1.0%,"22,734",1.4%
PG-US_12/31/2017,"13,359",-10.5%,64.7%,17.2%,36.3%,"22,414",-26.8%
PG-US_12/31/2016,"14,922",2.8%,47.5%,-0.1%,-0.1%,"30,633",-5.4%
PG-US_12/31/2015,"14,513",-6.7%,47.6%,0.0%,0.0%,"32,370",0.0%


## R&D Focused

In [118]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['Sales','RD','FCF', 'FCF_ROIC', 'FCF_RD_ROIC', 'ROA',
'Cash','LT_Debt','TangibleCapital','RD_Cap','Capex']]

### 'CurrAssets','ActPay','NetFixed',

,Sales,RD,FCF,FCF_ROIC,FCF_RD_ROIC,ROA,Cash,LT_Debt,TangibleCapital,RD_Cap,Capex
Instance,,,,,,,,,,,
PG-US_5/6/2022,"79,618",0,"13,953",64.9%,64.9%,12.1%,"10,182","22,594","21,488",0.0%,"3,178"
PG-US_12/31/2021,"77,138",0,"15,247",70.9%,70.9%,11.9%,"10,595","21,683","21,504",0.0%,"3,028"
PG-US_12/31/2020,"72,470",0,"15,129",67.6%,0.0%,11.6%,"12,811","22,938","22,365",0.0%,"2,844"
PG-US_12/31/2019,"68,792",0,"12,498",54.5%,54.5%,3.6%,"10,383","21,015","22,950",0.0%,"3,346"
PG-US_12/31/2018,"66,869",0,"11,138",49.0%,49.0%,8.3%,"14,349","21,566","22,734",0.0%,"3,665"
PG-US_12/31/2017,"65,193",0,"9,527",42.5%,42.5%,8.5%,"14,731","17,830","22,414",0.0%,"3,832"
PG-US_12/31/2016,"65,290",0,"11,456",37.4%,37.4%,7.8%,"14,369","18,996","30,633",0.0%,"3,466"
PG-US_12/31/2015,"72,620",0,"11,055",34.2%,34.2%,6.7%,"12,407","17,803","32,370",0.0%,"3,458"


## Growth Focused

In [119]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['EBITGrowth-1y', 'EBITGrowth-3y', 'SalesGrowth-1y','SalesGrowth-3y', 
'OCFGrowth-1y','OCFGrowth-3y',]]

,EBITGrowth-1y,EBITGrowth-3y,SalesGrowth-1y,SalesGrowth-3y,OCFGrowth-1y,OCFGrowth-3y
Instance,,,,,,
PG-US_5/6/2022,1.1%,29.2%,6.3%,18.7%,-10.1%,12.0%
PG-US_12/31/2021,12.0%,30.5%,6.4%,15.4%,1.7%,23.5%
PG-US_12/31/2020,11.1%,17.0%,5.3%,11.2%,13.4%,34.5%
PG-US_12/31/2019,4.9%,5.1%,2.9%,5.4%,7.0%,6.2%
PG-US_12/31/2018,0.5%,-5.3%,2.6%,-7.9%,10.8%,2.0%
PG-US_12/31/2017,-0.3%,-11.5%,-0.1%,-22.2%,-10.5%,-14.1%
PG-US_12/31/2016,-5.5%,-7.3%,-10.1%,-22.9%,2.8%,5.5%
PG-US_12/31/2015,-6.1%,-5.6%,-13.3%,-12.8%,-6.7%,4.5%


In [120]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['RD','FCF', 'FCF_ROIC', 'FCF_RD_ROIC','RD_Cap']]

,RD,FCF,FCF_ROIC,FCF_RD_ROIC,RD_Cap
Instance,,,,,
PG-US_5/6/2022,0,"13,953",64.9%,64.9%,0.0%
PG-US_12/31/2021,0,"15,247",70.9%,70.9%,0.0%
PG-US_12/31/2020,0,"15,129",67.6%,0.0%,0.0%
PG-US_12/31/2019,0,"12,498",54.5%,54.5%,0.0%
PG-US_12/31/2018,0,"11,138",49.0%,49.0%,0.0%
PG-US_12/31/2017,0,"9,527",42.5%,42.5%,0.0%
PG-US_12/31/2016,0,"11,456",37.4%,37.4%,0.0%
PG-US_12/31/2015,0,"11,055",34.2%,34.2%,0.0%


In [121]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['ShareholderYield1','DividendYield','DownsideBeta', 'UpsideBeta']]

,ShareholderYield1,DividendYield,DownsideBeta,UpsideBeta
Instance,,,,
PG-US_5/6/2022,4.4%,2.3%,0.34,0.42
PG-US_12/31/2021,4.6%,2.1%,0.46,0.36
PG-US_12/31/2020,2.8%,2.2%,0.63,0.87
PG-US_12/31/2019,2.2%,2.3%,0.59,0.47
PG-US_12/31/2018,4.9%,3.1%,0.51,0.44
PG-US_12/31/2017,8.4%,3.0%,0.35,0.37
PG-US_12/31/2016,4.8%,3.2%,0.49,0.65
PG-US_12/31/2015,2.6%,3.3%,0.77,0.78


In [122]:
roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['EBIT_EV','OCF_EV','FCF_EV','EBIT_RD_EV','OCF_RD_EV','FCF_RD_EV']]

,EBIT_EV,OCF_EV,FCF_EV,EBIT_RD_EV,OCF_RD_EV,FCF_RD_EV
Instance,,,,,,
PG-US_5/6/2022,4.6%,4.3%,3.5%,4.6%,4.3%,3.5%
PG-US_12/31/2021,4.5%,4.4%,3.6%,4.4%,4.4%,3.6%
PG-US_12/31/2020,4.7%,4.9%,4.1%,0.0%,0.0%,0.0%
PG-US_12/31/2019,4.6%,4.8%,3.8%,4.6%,4.8%,3.8%
PG-US_12/31/2018,5.8%,5.9%,4.4%,5.8%,5.9%,4.4%
PG-US_12/31/2017,5.8%,5.3%,3.8%,5.8%,5.3%,3.8%
PG-US_12/31/2016,6.3%,6.4%,4.9%,6.3%,6.4%,4.9%
PG-US_12/31/2015,6.6%,6.2%,4.7%,6.6%,6.2%,4.7%


In [123]:
df = roic_pretty[roic_pretty.Symbol == lookup_symbol]\
.sort_values('StartDate', ascending=False)\
[['EBIT_EV','OCF_EV','FCF_EV','EBIT_RD_EV','OCF_RD_EV','FCF_RD_EV']]

In [124]:
# from IPython.display import display, HTML, Latex, TextDisplayObject

In [125]:
# html = (
#     df.style
#     .format(format_dict)
#     .set_properties(**{'font-size': '9pt', 'font-family': 'Calibri'})
#     .render()
# )


In [126]:
# dft.to_html(classes=["table-bordered", "table-striped", "table-hover", "isi"]

# I make "isi" class and write it in CSS like this :

# .isi {
# text-align:center; }


In [127]:
# display(HTML(roic_pretty[(roic_pretty.Symbol==lookup_symbol)]\
# .sort_values('StartDate', ascending=False)\
# [['MarketCap']].to_html()))

<a id='List'></a>

### Output a list of companies

In [128]:
factor1 = 'EBIT_ROIC'
factor2 = 'EBIT_EV'
factor3 = 'DividendYield'

instance_list = ['MO-US_12/31/2020', 'HRB-US_12/31/2020', 'OMC-US_12/31/2020',
       'GILD-US_12/31/2020', 'PM-US_12/31/2020', 'KHC-US_12/31/2020',
       'STX-US_12/31/2020', 'HPQ-US_12/31/2020', 'CSCO-US_12/31/2020',
       'GIS-US_12/31/2020', 'STRA-US_12/31/2020', 'ABBV-US_12/31/2020',
       'CAH-US_12/31/2020', 'IPG-US_12/31/2020', 'SJM-US_12/31/2020',
       'HBI-US_12/31/2020', 'NTAP-US_12/31/2020', 'CPB-US_12/31/2020',
       'BBY-US_12/31/2020', 'KMB-US_12/31/2020', 'INTC-US_12/31/2020',
       'MRK-US_12/31/2020', 'LMT-US_12/31/2020', 'VZ-US_12/31/2020',
       'TAP-US_12/31/2020', 'FOX-US_12/31/2020', 'CAG-US_12/31/2020',
       'CVS-US_12/31/2020', 'FOXA-US_12/31/2020', 'DGX-US_12/31/2020']

In [129]:
list_of_symbols = ['AMZN-US','MA-US','V-US','MCO-US','NVDA-US','CDNS-US','DPZ-US',
'MSCI-US','GOOG-US','BRK.B-US','MSFT-US','SPGI-US','MNST-US','AAPL-US','FDS-US',
'BKNG-US','ANSS-US','ADBE-US','ETSY-US','PYPL-US','ISRG-US','SHOP-US','NOW-US',
'SNPS-US','REGN-US','CRM-US','TSLA-US']

In [130]:
## remove the date portion from the instance values, leaving just the symbol. 

list_of_symbols = []

for i in (instance_list):
#     print(i)
    i = i.rsplit('_12/31/2020', 1)[0]
    list_of_symbols.append(i)
#     print(i)

In [131]:
list_of_symbols = ['TSLA-US', 'TDOC-US', 'COIN-US', 'U-US', 'ROKU-US', 'SQ-US', 'PATH-US', 'SHOP-US', 
                   'ZM-US', 'TWLO-US', 'EXAS-US', 'SPOT-US', 'Z-US', 'PLTR-US', 'CRSP-US', 'NTLA-US', 
                   'TWTR-US', 'DKNG-US', 'BEAM-US', 'FATE-US', 'NVTA-US', 'PACB-US', 'DOCU-US']

In [132]:
list_of_symbols = ['F-US','TSLA-US','GM-US']
list_of_symbols = ['ETSY-US','NOW-US','CRM-US']

In [133]:
list_of_symbols = ['AMZN-US','MA-US','V-US','MCO-US','NVDA-US','CDNS-US','DPZ-US',
'MSCI-US','GOOG-US','BRK.B-US','MSFT-US','SPGI-US','MNST-US','AAPL-US','FDS-US',
'BKNG-US','ANSS-US','ADBE-US','ETSY-US','PYPL-US','ISRG-US','SHOP-US','NOW-US',
'SNPS-US','REGN-US','CRM-US','TSLA-US']

In [134]:
print(list_of_symbols)

['AMZN-US', 'MA-US', 'V-US', 'MCO-US', 'NVDA-US', 'CDNS-US', 'DPZ-US', 'MSCI-US', 'GOOG-US', 'BRK.B-US', 'MSFT-US', 'SPGI-US', 'MNST-US', 'AAPL-US', 'FDS-US', 'BKNG-US', 'ANSS-US', 'ADBE-US', 'ETSY-US', 'PYPL-US', 'ISRG-US', 'SHOP-US', 'NOW-US', 'SNPS-US', 'REGN-US', 'CRM-US', 'TSLA-US']


In [135]:
# df1 = roic_pretty[roic_pretty.Symbol == lookup_symbol]\
#     .sort_values('StartDate', ascending=False)\
#     [['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA',
#     'CurrAssets','Cash','LT_Debt','ActPay','NetFixed', 'TangibleCapital']]

In [136]:
# list_of_symbols

In [137]:
import datetime
from datetime import timedelta

## get today and yesterdays dates, nicely formatted
now = datetime.datetime.now()
yesterday = yesterday = datetime.datetime.now() - timedelta(days=1)
todays_date = now.strftime("%Y_%m_%d_%H%M")

In [138]:
todays_date

'2022_05_18_2029'

In [139]:
## stop here because maybe you don't want to output the png files yet

print blah

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(blah)? (2641694686.py, line 3)

In [ ]:
### make a directory today's date and time, you'll put the png files there

dir_prefix = 'Symbols/'

## make a new sub directory there with todays date if it doesn't exist already

import os
if not os.path.exists(f"{dir_prefix}{todays_date}"):
    os.makedirs(f"{dir_prefix}{todays_date}")

In [ ]:
### go through a list of symbols and make each dataframe into a png file

import dataframe_image as dfi

for lookup_symbol in (list_of_symbols):
    df1 = roic_pretty[roic_pretty.Symbol == lookup_symbol]\
    .sort_values('StartDate', ascending=False)\
    [['OCF', 'OCFGrowth-1y', 'OCF_ROIC', 'OCF_ROIC_Delta', 'OCF_ROIC_rel', 'TangibleCapital', 'Capital_Growth']]
    dfi.export(df1, f"{dir_prefix}{todays_date}/{lookup_symbol}.png")

In [ ]:
# ### go through a list of symbols and make each dataframe into a png file

# import dataframe_image as dfi

# for lookup_symbol in (list_of_symbols):
#     df1 = roic_pretty[roic_pretty.Symbol == lookup_symbol]\
#     .sort_values('StartDate', ascending=False)\
#     [['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA',
#     'CurrAssets','Cash','LT_Debt','ActPay','NetFixed', 'TangibleCapital']]
#     dfi.export(df1, f"{dir_prefix}{todays_date}/{lookup_symbol}.png")

In [ ]:
# print('ocf',3019 + 1641 + 2124 + 3147)

# print('fcf',1868 + 293 + 619 + 1328)

In [ ]:
# print('avg cash 4 quarters', 17205)

In [ ]:
# 25002 - 16065 - 8260 + 17298

In [ ]:
# 9931/17975

In [ ]:
### write a for loop for showing a list of companies at once
### just loop through a list of companies

for lookup_symbol in (list_of_symbols):
    roic_pretty[roic_pretty.Symbol == lookup_symbol]\
    .sort_values('StartDate', ascending=False)\
    [['Sales','EBIT', 'EBIT_ROIC', 'OCF', 'OCF_ROIC','ROA',
    'CurrAssets','Cash','LT_Debt','ActPay','NetFixed', 'TangibleCapital']]

<a id='R&D'></a>

#### R&D Focused

In [ ]:
#### R&D Focused

### write a for loop for showing a list of companies at once
### just loop through a list of companies

for lookup_symbol in (list_of_symbols):
    roic_pretty[roic_pretty.Symbol == lookup_symbol]\
    .sort_values('StartDate', ascending=False)\
    [['Sales', 'OCF', 'OCF_ROIC', 'OCF_RD','OCF_RD_ROIC', 'RD_Cap'
    'CurrAssets','Cash','LT_Debt','ActPay','NetFixed', 'TangibleCapital']]

<a id="Plotting"></a>

### Plotting

In [ ]:
def create_delta_plot(orig_df, symbol=''):
    plotdf = orig_df[orig_df.Symbol == symbol]\
            .sort_values('StartDate', ascending=True)\
            [['StartDate','Symbol','OCF', 'OCFGrowth1year', 'OCF_ROIC', 'OCF_ROIC_Delta', 'OCF_ROIC_rel', 
              'TangibleCapital', 'Capital_Growth']]

    plotdf['StartDate'] = pd.to_datetime(plotdf.StartDate)
    plotdf['year'] = plotdf.StartDate.dt.year
    plotdf['color'] = np.where(plotdf['OCF_ROIC_rel']>=0, plot_settings.color_list[0], plot_settings.color_list[1])
    
    fig = make_subplots(rows=3, cols=1,
                       shared_xaxes=True,
                       vertical_spacing=0.12,
                       specs=[[{}], 
                             [{'rowspan':2}],
                             [None]])

    # top line graph
    fig.add_trace(go.Scatter(x=plotdf['year'], 
                             y=plotdf['OCF_ROIC'],
                             line=dict(color=plot_settings.color_list[2]),
                             marker=dict(size=10, 
    #                                      color='rgba(134,134,134,.2)',
                                         color="#ffffff",
                                         line=dict(width=2.5,
                                                   color=plot_settings.color_list[2]))), 
                  row=1, col=1)

    # bottom bar plot
    fig.add_trace(go.Bar(x=plotdf['year'], 
                         y=plotdf['OCF_ROIC_rel'],
                         marker_color=plotdf['color'],
                         opacity=.8,
                         text=plotdf['OCF_ROIC_rel'],
                         textposition='inside',
                         texttemplate='%{text:,.0%}',
                         textfont=dict(size=11)), 
                  row=2, col=1)

    fig.update_layout(template=plot_settings.dockstreet_template,
                      showlegend=False,
                      uniformtext_minsize=8, 
                      uniformtext_mode='hide',
                      title=dict(text=f'<b>{symbol} Annual ' + '<span style="color:#868686;">OCF ROIC</span>' + \
                                 ' values and YoY changes</b>',
                                 font_size=22,
                                 x=0.04,
                                 y=0.98),
                      font_family="Avenir",
                      font_color="#4c4c4c",
                      font_size=14)

    # -------
    # FORMAT Y-AXIS RANGE FOR TOP HALF
    topy_min = min(plotdf['OCF_ROIC'])-.2
    topy_max = max(plotdf['OCF_ROIC'])+.2
    
#     topy_min_rnd, topy_max_rnd, tickstep, tickvals = ytick_setup(topy_min, topy_max)
    
#     fig.update_yaxes(range=[topy_min_rnd, topy_max_rnd],
# #                      nticks=3,
# #                      tickmode='array',
# #                      tickvals=tickvals,
#                      tickmode='linear',
#                      tick0=topy_min_rnd,
#                      dtick=tickstep/2,
#                      ticklabelstep=2,
#                      row=1, col=1)
    
    likely_int = int(math.ceil(((topy_max - topy_min)/4 + .125) / .25)) * .25
    likely_int = round(((topy_max - topy_min)/4 + .125) / .25) * .25
    likely_tick_cnt = round((topy_max - topy_min)/likely_int) + 1

    if likely_tick_cnt <= 4:
        dtick = likely_int/2
        ticklabelstep = 2
        tick0 = math.floor((topy_min + .2) / (likely_int/2)) * (likely_int/2)
    else:
        dtick = likely_int
        ticklabelstep = 1
        
    fig.update_yaxes(range=[topy_min, topy_max],
                 dtick=dtick,
                 ticklabelstep=ticklabelstep,
#                  tick0 = tick0,
                 row=1, col=1)

    # -------
    # FORMAT Y-AXIS RANGE FOR BOTTOM HALF
    boty_min = min(plotdf['OCF_ROIC_rel'])
    boty_max = max(plotdf['OCF_ROIC_rel'])#+.055
    
    # if min value is a negative, extend y axis to 10% lower than min value, otherwise will start at 0%
    if boty_min < 0:
        boty_min = boty_min + (boty_min*.05)
    
    # add 7% of max value to yaxis range
    boty_max = boty_max + (boty_max*.07)

    fig.update_yaxes(range=[boty_min, boty_max],
                     row=2, col=1)
    
    # -------    
    fig.update_yaxes(tickformat=",.0%")

    fig.update_xaxes(showgrid=False, 
                     row=1, col=1)
    fig.update_xaxes(type='category')

    fig.add_annotation(x=-.35, 
                       y=topy_max, #1.7, 
                       xref='x', 
                       yref='y', 
                       xanchor='left', 
                       yanchor='bottom',
                       align='left',
                       text='<b>Annual ' + '<span style="color:#868686;">OCF ROIC</span>' + \
                       ' values</b>',
                       showarrow=False, 
                       font_size=14, 
                       col=1, row=1)

    fig.add_annotation(x=-.35, 
                       y= boty_max, 
                       xref='x', 
                       yref='y', 
                       xanchor='left', 
                       yanchor='bottom',
                       align='left',
                       text='<b>Annual ' + '<span style="color:#0173c2;">positive</span>' + ' and ' + \
                       '<span style="color:#cd534c;">negative</span>' + ' YoY % changes in OCF ROIC</b>',
                       showarrow=False, 
                       font_size=14, 
                       col=1, row=2)

    fig.update_layout(height=630)

    fig.show()

In [ ]:
def plotting(symbol_instance):
    
    plotdf = roic_with_deltas[roic_with_deltas.Symbol == symbol_instance]\
        .sort_values('StartDate', ascending=True)\
        [['StartDate','Symbol','OCF', 'OCFGrowth1year', 'OCF_ROIC', 'OCF_ROIC_Delta', 'OCF_ROIC_rel', 
          'TangibleCapital', 'Capital_Growth']]

    plotdf['StartDate'] = pd.to_datetime(plotdf.StartDate)
    plotdf['year'] = plotdf.StartDate.dt.year
    plotdf['color'] = np.where(plotdf['OCF_ROIC_rel']>=0, plot_settings.color_list[0], plot_settings.color_list[1])
    
    fig = make_subplots(rows=3, cols=1,
                   shared_xaxes=True,
                   vertical_spacing=0.1,
                   specs=[[{}], 
                         [{'rowspan':2}],
                         [None]])

    # top line graph
    fig.add_trace(go.Scatter(x=plotdf['year'], 
                         y=plotdf['OCF_ROIC'],
                         line=dict(color=plot_settings.color_list[2]),
                         marker=dict(size=10, 
#                                      color='rgba(134,134,134,.2)',
                                     color="#ffffff",
                                     line=dict(width=2.5,
                                               color=plot_settings.color_list[2]))), 
              row=1, col=1)

    # bottom bar plot
    fig.add_trace(go.Bar(x=plotdf['year'], 
                     y=plotdf['OCF_ROIC_rel'],
                     marker_color=plotdf['color'],
                     opacity=.8,
                     text=plotdf['OCF_ROIC_rel'],
                     textposition='inside',
                     texttemplate='%{text:,.0%}',
                     textfont=dict(size=11)), 
              row=2, col=1)

    fig.update_layout(template=plot_settings.dockstreet_template,
                  showlegend=False,
                  uniformtext_minsize=8, 
                  uniformtext_mode='hide',
                  title=dict(text=f'<b>{symbol_instance} : Annual ' + '<span style="color:#868686;">OCF ROIC</span>' + \
                             ' values and YoY changes</b>',
                             font_size=22,
                             x=0.04),
                  font_family="Avenir",
                  font_color="#4c4c4c",
                  font_size=14)

    fig.update_yaxes(tickformat=",.0%")

    topy_min = min(plotdf['OCF_ROIC'])-.2
    topy_max = max(plotdf['OCF_ROIC'])+.2

    likely_int = int(math.ceil(((topy_max - topy_min)/4 + .125) / .25)) * .25
    likely_int = round(((topy_max - topy_min)/4 + .125) / .25) * .25
    likely_tick_cnt = round((topy_max - topy_min)/likely_int) + 1

    if likely_tick_cnt <= 4:
        dtick = likely_int/2
        ticklabelstep = 2
        tick0 = math.floor((topy_min + .2) / (likely_int/2)) * (likely_int/2)
    else:
        dtick = likely_int
        ticklabelstep = 1

    boty_min = min(plotdf['OCF_ROIC_rel'])-.1
    boty_max = max(plotdf['OCF_ROIC_rel'])+.1


    fig.update_yaxes(range=[topy_min, topy_max],
                     dtick=dtick,
                     ticklabelstep=ticklabelstep,
    #                  tick0 = tick0,
                     row=1, col=1)
    fig.update_yaxes(range=[boty_min, boty_max+(boty_max*.1)],
                     row=2, col=1)

    fig.update_xaxes(showgrid=False, 
                     row=1, col=1)
    fig.update_xaxes(type='category')

    fig.add_annotation(x=-.35, 
                       y=topy_max, #1.7, 
                       xref='x', 
                       yref='y', 
                       xanchor='left', 
                       align='left',
                       text='<b>Annual ' + '<span style="color:#868686;">OCF ROIC</span>' + \
                       ' values</b>',
                       showarrow=False, 
                       font_size=14, 
                       col=1, row=1)

    fig.add_annotation(x=-.35, 
                       y= boty_max+(boty_max*.1), 
                       xref='x', 
                       yref='y', 
                       xanchor='left', 
                       align='left',
                       text='<b>Annual ' + '<span style="color:#0173c2;">positive</span>' + ' and ' + \
                       '<span style="color:#cd534c;">negative</span>' + ' YoY % changes in OCF ROIC</b>',
                       showarrow=False, 
                       font_size=14, 
                       col=1, row=2)

    fig.update_layout(height=600)

    fig.show()
    fig.write_image(f"{symbol_instance}.pdf")

In [ ]:
# list_of_symbols = ['AMZN-US','MA-US','V-US','MCO-US','NVDA-US','CDNS-US','DPZ-US',
# 'MSCI-US','GOOG-US','BRK.B-US','MSFT-US','SPGI-US','MNST-US','AAPL-US','FDS-US',
# 'BKNG-US','ANSS-US','ADBE-US','ETSY-US','PYPL-US','ISRG-US','NOW-US',
# 'SNPS-US','REGN-US','CRM-US','TSLA-US','SHOP-US',]

In [ ]:
list_of_symbols = ['NET-US', 'TSLA-US','CRWD-US','TTD-US','UPWK-US','APPN-US','PINS-US',
                   'ROKU-US','FSLY-US','SFIX-US','NFLX-US']

In [ ]:
### loop 

for holding in list_of_symbols:
    create_delta_plot(roic_with_deltas, symbol=holding)

In [ ]:
list_of_symbols = ['KO-US','PG-US','KMB-US','GIS-US','CL-US','K-US','CPB-US','KHC-US',
                  'SJM-US','CAG-US','CLX-US']

In [ ]:
for holding in list_of_symbols:
    plotting(holding)

## Summary plot with sorted list

In [ ]:
# summary - sorted list
# chgsort = pd.read_csv('/Users/Evan/Downloads/list_of_ten_2020.csv', na_values=['#VALUE!', '#DIV/0!', '#SPILL!'])

In [ ]:
# year_choice = 2020

In [ ]:
roic_with_deltas['StartDate'] = pd.to_datetime(roic_with_deltas['StartDate'])
plotdfsum = roic_with_deltas[roic_with_deltas['StartDate']==datetime.datetime(year_choice,12,31)]
plotdfsum = plotdfsum[(plotdfsum.FCF_RD_ROIC>0.8) & (plotdfsum.MarketCap>7000)\
                     & (plotdfsum.EBITMargin>.05) & (plotdfsum.ROA>0)]
plotdfsum = plotdfsum.sort_values(by='EBIT_ROIC_rel', ascending=False)
plotdfsum['relrank'] = plotdfsum['EBIT_ROIC_rel'].rank(axis=0, ascending=False)
plotdfsum = plotdfsum[['StartDate','Symbol','Company_Name','Total_Return','FCF_RD_ROIC','EBIT_ROIC_rel','relrank']]

In [ ]:
summ = go.Figure()

summ.add_trace(go.Bar(x=chgsort['FCF_RD_ROIC'],
                      y=chgsort['Symbol'],
                      name='ROIC',
                      orientation='h',
                      width=0.38,
                      marker_color='rgba(205,83,76,0.5)',#'rgba(134,134,134,0.5)',#plot_settings.color_list[0],
                      showlegend=True,
                      legendgroup='roic'))

summ.add_trace(go.Scatter(x=chgsort['EBIT_ROIC_rel'], 
                          y=chgsort['Symbol'],
                          name='Chg in ROIC',
                          marker=dict(size=17,
                                      color='rgba(1,115,194,0.9)',#plot_settings.color_list[0],
                                      ),
                          mode='markers',
                          showlegend=True,
                          legendgroup='chg'))

summ.add_trace(go.Bar(x=chgsort['EBIT_ROIC_rel'],
                      y=chgsort['Symbol'],
                      orientation='h',
                      width=.07,
                      marker_color='rgba(1,115,194,0.6)',#plot_settings.color_list[0],
                      showlegend=False,
                      legendgroup='chg'))

summ.update_yaxes(showgrid=False,
                 autorange='reversed',)
summ.update_xaxes(zeroline=False, 
                  tickformat=",.0%")

summ.update_layout(template=plot_settings.dockstreet_template,
                   barmode='overlay',
                   legend=dict(orientation='h', 
                               y=1.0, 
                               x=.0, 
                               xanchor='left',
                               yanchor='bottom',
                              font=dict(size=12)),
                    title=dict(text='<b>2020: ' + \
                                f'<span style="color:rgba(205,83,76,0.8);">ROIC</span>' + \
                                ' and ' + \
                                f'<span style="color:{plot_settings.color_list[0]};">YoY \u0394 ROIC</span>' + \
                               ' for 10 tickers with highest YoY % changes</b>',
                                 font_size=22,
                                 x=0.02,
                                 y=0.94),
                    font_family="Avenir",
                    font_color="#4c4c4c",
                    font_size=14)

summ.show()

In [ ]:
summ = go.Figure()

summ.add_trace(go.Scatter(x=chgsort['EBIT_ROIC_rel'],
                          y=[chgsort['Symbol'], ['\32']*len(chgsort['Symbol'])],
                          name='Chg in ROIC',
                          mode='markers',
                          marker_size=10,
                          marker_color=plot_settings.color_list[0],
                          showlegend=True,
                          legendgroup='chg'))

summ.add_trace(go.Bar(x=chgsort['EBIT_ROIC_rel'],
                      y=[chgsort['Symbol'], ['\32']*len(chgsort['Symbol'])],
                      name='Chg in ROIC',
                      orientation='h',
                      width=.05,
                      marker_color='rgba(1,115,194,0.8)',#plot_settings.color_list[0],
                      showlegend=False,
                      legendgroup='chg'))

summ.add_trace(go.Scatter(x=chgsort['FCF_RD_ROIC'],
                          y=[chgsort['Symbol'], ['\32\32']*len(chgsort['Symbol'])],
                          name='ROIC',
                          mode='markers',
                          marker_size=10,
                          marker_color=plot_settings.color_list[1],
                          showlegend=True,
                          legendgroup='roic'))

summ.add_trace(go.Bar(x=chgsort['FCF_RD_ROIC'],
                      y=[chgsort['Symbol'], ['\32\32']*len(chgsort['Symbol'])],
                      name='ROIC',
                      orientation='h',
                      width=.05,
                      marker_color='rgba(205,83,76,0.8)',#plot_settings.color_list[1],
                      showlegend=False,
                      legendgroup='roic'))

summ.update_yaxes(showgrid=False, 
                  autorange='reversed',
                  dividerwidth=0.3,
                  dividercolor='rgba(189,189,189,0.5)',#'#cacaca',
                  showdividers=True,)

summ.update_xaxes(zeroline=False, 
                  tickformat=",.0%")

summ.update_layout(template=plot_settings.dockstreet_template,
                   height=650,
                   legend=dict(orientation='h', 
                               y=1.0, 
                               x=.0, 
                               xanchor='left',
                               yanchor='bottom'),
                   title=dict(text='<b>2020: Ten tickers with highest ' + f'<span style="color:{plot_settings.color_list[0]};">OCF ROIC</span>' + \
                                 ' values and YoY changes</b>',
                                 font_size=22,
                                 x=0.04,
                                 y=0.98),
                   font_family="Avenir",
                   font_color="#4c4c4c",
                   font_size=14)

summ.show()

[Jump to Final Output](#Final_Output)<br>
[Jump to List of Companies](#List)<br>
[R&D Multiple Companies](#R&D)<br>
[Plotting](#Plotting)